<div style='float: right'><img src='pic/hitori.png'/></div>

## <div id='hitori' />ひとりにしてくれ

In [ ]:
#!pip3 install pulp ortoolpy
import numpy as np
from itertools import chain, product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars, unionfind
data = np.array([list(s) for s in """\
18626753
31118222
83247651
37583314
54467182
71432535
22834475
22314465""".split()]).astype(int)
nn = len(data)

### 問題
* 盤面に並んでいる数字のうちいくつかを黒くぬり、タテでもヨコでも同じ列に同じ数字が複数個入らないようにします
* 黒マスをタテヨコに連続したり、黒マスで盤面を分断してはいけません

### 変数
* v：0:number, 1:black (1)

### 制約
* 黒以外の数字は縦横に重複しないこと (2)
* 黒は連続しないこと (3)
* 黒で分断しないこと (4)

In [ ]:
m = LpProblem()
v = np.array(addbinvars(nn, nn)) # 0:number, 1:black (1)
for i, j in product(range(nn), range(nn)):
    for x in [v[i,:][data[i,:] == j+1], v[:,i][data[:,i] == j+1]]:
        m += lpSum(x) >= len(x)-1 # (2)
for e in chain((v[1:,:] + v[:-1,:]).flat, (v[:,1:] + v[:,:-1]).flat):
    m += e <= 1 # (3)
while True:
    %time m.solve()
    r = np.vectorize(value)(v)
    if unionfind.isconnected(1-r):
        break
    m += lpSum(v[r==1]) <= r.sum() - 1 # (4)
data[r==1] = 0
data.tolist()